---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

In [5]:
def graph_identification():
    
    # Your Code Here
    res = [(nx.average_clustering(i),nx.average_shortest_path_length(i)) for i in P1_Graphs]
    res = ['PA','SW_L','SW_L','PA','SW_H']
    return res # Your Answer Here ['PA','SW_S','SW_S','PA','SW_L']

In [6]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [7]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [8]:
def salary_predictions():
    
    # Your Code Here
    
    from sklearn.preprocessing import MinMaxScaler
    
    
    
    df = pd.DataFrame(index= G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G,'Department'))
    df["ManagementSalary"] = pd.Series(nx.get_node_attributes(G,'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df['pr'] = pd.Series(nx.pagerank(G))

    df_test = df[(df.ManagementSalary.isnull() == True)]
    df = df.dropna()
    y = df.iloc[:,1]
    X = df.drop('ManagementSalary',axis=1)
    
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    df_test = df_test.dropna(axis = 1)
    X_test = scaler.transform(df_test)
    
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import roc_auc_score
    
    param_grid= {'max_depth':[2,3,4,5,6,7],'learning_rate':[0.001,0.01,0.1]}
    
    g=GridSearchCV(GradientBoostingClassifier(random_state = 0),param_grid=param_grid,cv= 5,scoring = 'roc_auc')
    g.fit(X_scaled,y)
    z = g.predict_proba(X_test)[:,1]
    
    res = pd.Series(z,index=df_test.index)
    
    return res #Your Answer Here

In [9]:
salary_predictions()

1       0.013799
2       0.987366
5       0.981083
8       0.168930
14      0.063598
18      0.026365
27      0.057571
30      0.704002
31      0.180140
34      0.020812
37      0.021655
40      0.026060
45      0.021841
54      0.281180
55      0.610160
60      0.123533
62      0.981083
65      0.982742
77      0.019837
79      0.024754
97      0.016149
101     0.002005
103     0.678357
108     0.041873
113     0.134702
122     0.001184
141     0.455744
142     0.981083
144     0.019835
145     0.701481
          ...   
913     0.015866
914     0.015084
915     0.001059
918     0.039544
923     0.045223
926     0.043846
931     0.012620
934     0.001090
939     0.000927
944     0.001090
945     0.045223
947     0.045062
950     0.041052
951     0.012478
953     0.001959
959     0.000927
962     0.001751
963     0.118436
968     0.054206
969     0.054206
974     0.027559
984     0.001090
987     0.034203
989     0.045062
991     0.069941
992     0.000927
994     0.001090
996     0.0010

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [15]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [26]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
def new_connections_predictions():
    for node in G.nodes():
        G.node[node]['community'] = G.node[node]['Department']
    preferential_attachment = list(nx.preferential_attachment(G))
    df = pd.DataFrame(index=[(x[0], x[1]) for x in preferential_attachment])
    df['preferential_attachment'] = [x[2] for x in preferential_attachment]
    cn_soundarajan_hopcroft = list(nx.cn_soundarajan_hopcroft(G))
    df_cn_soundarajan_hopcroft = pd.DataFrame(index=[(x[0], x[1]) for x in cn_soundarajan_hopcroft])
    df_cn_soundarajan_hopcroft['cn_soundarajan_hopcroft'] = [x[2] for x in cn_soundarajan_hopcroft]
    df = df.join(df_cn_soundarajan_hopcroft,how='outer')
    df['cn_soundarajan_hopcroft'] = df['cn_soundarajan_hopcroft'].fillna(value=0)
    df['resource_allocation_index'] = [x[2] for x in list(nx.resource_allocation_index(G))]
    df['jaccard_coefficient'] = [x[2] for x in list(nx.jaccard_coefficient(G))]
    df = future_connections.join(df,how='outer')
    df_train = df[~pd.isnull(df['Future Connection'])]
    df_test = df[pd.isnull(df['Future Connection'])]
    features = ['cn_soundarajan_hopcroft', 'preferential_attachment', 'resource_allocation_index', 'jaccard_coefficient']
    X_train = df_train[features]
    Y_train = df_train['Future Connection']
    X_test = df_test[features]
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = MLPClassifier(hidden_layer_sizes = [10, 5], alpha = 5,
                       random_state = 0, solver='lbfgs', verbose=0)
    clf.fit(X_train_scaled, Y_train)
    test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    predictions = pd.Series(test_proba,X_test.index)
    target = future_connections[pd.isnull(future_connections['Future Connection'])]
    target['prob'] = [predictions[x] for x in target.index]
    return target['prob']

In [27]:
new_connections_predictions()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(107, 348)    0.030798
(542, 751)    0.012661
(20, 426)     0.590189
(50, 989)     0.012774
(942, 986)    0.012896
(324, 857)    0.012758
(13, 710)     0.157992
(19, 271)     0.107565
(319, 878)    0.012813
(659, 707)    0.012673
(49, 843)     0.012848
(208, 893)    0.012700
(377, 469)    0.008606
(405, 999)    0.021212
(129, 740)    0.018759
(292, 618)    0.023529
(239, 689)    0.012821
(359, 373)    0.010155
(53, 523)     0.076263
(276, 984)    0.012843
(202, 997)    0.012866
(604, 619)    0.047400
(270, 911)    0.012819
(261, 481)    0.063398
(200, 450)    0.997548
(213, 634)    0.012626
(644, 735)    0.041949
(346, 553)    0.012347
(521, 738)    0.011817
(422, 953)    0.019543
                ...   
(672, 848)    0.012819
(28, 127)     0.977766
(202, 661)    0.012300
(54, 195)     0.997570
(295, 864)    0.012741
(814, 936)    0.012643
(839, 874)    0.012896
(139, 843)    0.012750
(461, 544)    0.011453
(68, 487)     0.011390
(622, 932)    0.012713
(504, 936)    0.017593
(479, 528) 